#Codigo inicial de librerias , APIs y coordenadas

In [ ]:
#instalar los componenetes del servicio openrouteservice en el entorno de desarollo
!pip install openrouteservice folium deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
#instalar las librerias de necesarias para el proyecto
import openrouteservice
import folium
import random
from deap import base, creator, tools, algorithms
import time
import matplotlib.pyplot as plt

In [ ]:
#ingresar la API key que nos da la pagina web a la hora de resistrarnos Link:https://account.heigit.org/manage/key
api_key = "5b3ce3597851110001cf6248336a7e87ff1a448da4676b881d551923"
client = openrouteservice.Client(key=api_key)

In [ ]:
lugares = {
    "Inicio/Fin: Cra 16 A No 28B-33": (4.618317623364779, -74.07349813268002),
    "Colegio Distrital Usaquén": (4.7222, -74.0300),
    "Colegio INEM Kennedy": (4.6270, -74.1570),
    "Colegio Nicolás Esguerra": (4.6413, -74.1032),
    "Colegio Agustiniano Norte": (4.6936, -74.0502),
    "Colegio La Merced": (4.6500, -74.0650)
}

#Codigo de la solucion con Algoritmo genetico con TSP

In [ ]:
# Este codigo inicia un temporizador para poder calcular el tiempo en el cual se demora en ejecutar todo el codigo
start_time = time.time()

# Lista de colegios con coordenadas (lat, lon)
lugares = {
    "Inicio/Fin: Cra 16 A No 28B-33": (4.618317623364779, -74.07349813268002),
    "Colegio Distrital Usaquén": (4.7222, -74.0300),
    "Colegio INEM Kennedy": (4.6270, -74.1570),
    "Colegio Nicolás Esguerra": (4.6413, -74.1032),
    "Colegio Agustiniano Norte": (4.6936, -74.0502),
    "Colegio La Merced": (4.6500, -74.0650)
}

nombres = list(lugares.keys())
coordenadas = list(lugares.values())
N = len(coordenadas)

# OpenRouteService necesita (lon, lat), no (lat, lon)
coords_ors = [(lon, lat) for lat, lon in coordenadas]

# Matriz de distancias viales con OpenRouteService
dist_matrix = [[0]*N for _ in range(N)]
for i in range(N):
    for j in range(N):
        if i != j:
            try:
                res = client.directions([coords_ors[i], coords_ors[j]], profile='driving-car')
                distancia = res['routes'][0]['summary']['distance'] / 1000  # km
                dist_matrix[i][j] = distancia
            except:
                dist_matrix[i][j] = 9999  # Penalización por error

# Algoritmo genético TSP (basado en distancias reales)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(N), N)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluar(ind):
    return (sum(dist_matrix[ind[i]][ind[(i+1)%N]] for i in range(N)),)

toolbox.register("evaluate", evaluar)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.3)
toolbox.register("select", tools.selTournament, tournsize=3)

# Ejecutar
poblacion = toolbox.population(n=100)
algorithms.eaSimple(poblacion, toolbox, cxpb=0.7, mutpb=0.2, ngen=200, verbose=False)
mejor = tools.selBest(poblacion, 1)[0]
ruta_indices = mejor + [mejor[0]]

# Crear mapa
mapa = folium.Map(location=[4.65, -74.1], zoom_start=12)
for idx in ruta_indices:
    nombre = nombres[idx]
    lat, lon = coordenadas[idx]
    folium.Marker([lat, lon], tooltip=nombre).add_to(mapa)

# Agregar rutas reales entre puntos usando OpenRouteService
for i in range(len(ruta_indices) - 1):
    origen = coords_ors[ruta_indices[i]]
    destino = coords_ors[ruta_indices[i+1]]
    ruta = client.directions([origen, destino], profile='driving-car', format='geojson')
    folium.GeoJson(ruta, name="ruta").add_to(mapa)

# Resultado
distancia_total = evaluar(mejor)[0]
print(f"Distancia total optimizada por vías: {distancia_total:.2f} km")
mapa
end_time = time.time()
tiempo_total = end_time - start_time

Distancia total optimizada por vías: 52.66 km


In [ ]:
distancia_total = evaluar(mejor)[0]
print(f"Distancia total optimizada por vías: {distancia_total:.2f} km")
print(f"Tiempo total de ejecución: {tiempo_total:.2f} segundos")
print(f"🧬 Longitud de la ruta óptima: {len(ruta_indices) - 1} tramos")


Distancia total optimizada por vías: 40.19 km
Tiempo total de ejecución: 0.00 segundos
📏 Distancia total optimizada: 40.19 km
🧬 Longitud de la ruta óptima: 6 tramos


#Codigo de la solucion con Algoritmo del vecino más cercano

In [ ]:
import openrouteservice
import time

# API Key de OpenRouteService
api_key = "5b3ce3597851110001cf6248336a7e87ff1a448da4676b881d551923"
client = openrouteservice.Client(key=api_key)

# ⏱️ Tiempo de inicio
start_time = time.time()

# Coordenadas (lat, lon)
lugares = {
    "Inicio/Fin: Cra 16 A No 28B-33": (4.618317623364779, -74.07349813268002),
    "Colegio Distrital Usaquén": (4.7222, -74.0300),
    "Colegio INEM Kennedy": (4.6270, -74.1570),
    "Colegio Nicolás Esguerra": (4.6413, -74.1032),
    "Colegio Agustiniano Norte": (4.6936, -74.0502),
    "Colegio La Merced": (4.6500, -74.0650)
}

nombres = list(lugares.keys())
coordenadas = list(lugares.values())
N = len(coordenadas)

# Convertir a formato (lon, lat) para OpenRouteService
coords_ors = [(lon, lat) for lat, lon in coordenadas]

# Crear matriz de distancias
dist_matrix = [[0] * N for _ in range(N)]
for i in range(N):
    for j in range(N):
        if i != j:
            try:
                res = client.directions([coords_ors[i], coords_ors[j]], profile='driving-car')
                dist = res['routes'][0]['summary']['distance'] / 1000
                dist_matrix[i][j] = dist
            except:
                dist_matrix[i][j] = 9999

# Algoritmo del vecino más cercano
visitado = [False] * N
ruta = [0]  # empieza desde la posición 0 (inicio)
visitado[0] = True
actual = 0
total = 0

for _ in range(N - 1):
    menor_dist = float('inf')
    sig = -1
    for j in range(N):
        if not visitado[j] and dist_matrix[actual][j] < menor_dist:
            menor_dist = dist_matrix[actual][j]
            sig = j
    ruta.append(sig)
    visitado[sig] = True
    total += menor_dist
    actual = sig

# Volver al punto de inicio
ruta.append(0)
total += dist_matrix[actual][0]

# Mostrar resultados
print("🧭 Ruta óptima con algoritmo del vecino más cercano:\n")
for idx in ruta:
    print("→", nombres[idx])

print(f"\n📏 Distancia total aproximada: {total:.2f} km")

# ⏱️ Tiempo de ejecución
end_time = time.time()
print(f"\n⏱️ Tiempo de ejecución: {end_time - start_time:.2f} segundos")


🧭 Ruta óptima con algoritmo del vecino más cercano:

→ Inicio/Fin: Cra 16 A No 28B-33
→ Colegio La Merced
→ Colegio Nicolás Esguerra
→ Colegio INEM Kennedy
→ Colegio Agustiniano Norte
→ Colegio Distrital Usaquén
→ Inicio/Fin: Cra 16 A No 28B-33

📏 Distancia total aproximada: 59.66 km

⏱️ Tiempo de ejecución: 5.70 segundos


In [ ]:
!pip install openrouteservice folium

import openrouteservice
import folium
import time

# API Key de OpenRouteService
api_key = "5b3ce3597851110001cf6248336a7e87ff1a448da4676b881d551923"
client = openrouteservice.Client(key=api_key)

# ⏱️ Tiempo de inicio
start_time = time.time()

# Coordenadas (lat, lon)
lugares = {
    "Inicio/Fin: Cra 16 A No 28B-33": (4.618317623364779, -74.07349813268002),
    "Colegio Distrital Usaquén": (4.7222, -74.0300),
    "Colegio INEM Kennedy": (4.6270, -74.1570),
    "Colegio Nicolás Esguerra": (4.6413, -74.1032),
    "Colegio Agustiniano Norte": (4.6936, -74.0502),
    "Colegio La Merced": (4.6500, -74.0650)
}

nombres = list(lugares.keys())
coordenadas = list(lugares.values())
N = len(coordenadas)

# Convertir a formato (lon, lat) para OpenRouteService
coords_ors = [(lon, lat) for lat, lon in coordenadas]

# Crear matriz de distancias
dist_matrix = [[0] * N for _ in range(N)]
for i in range(N):
    for j in range(N):
        if i != j:
            try:
                res = client.directions([coords_ors[i], coords_ors[j]], profile='driving-car')
                dist = res['routes'][0]['summary']['distance'] / 1000
                dist_matrix[i][j] = dist
            except:
                dist_matrix[i][j] = 9999

# Algoritmo del vecino más cercano
visitado = [False] * N
ruta = [0]  # empieza desde la posición 0 (inicio)
visitado[0] = True
actual = 0
total = 0

for _ in range(N - 1):
    menor_dist = float('inf')
    sig = -1
    for j in range(N):
        if not visitado[j] and dist_matrix[actual][j] < menor_dist:
            menor_dist = dist_matrix[actual][j]
            sig = j
    ruta.append(sig)
    visitado[sig] = True
    total += dist_matrix[actual][sig]
    actual = sig

# Volver al punto de inicio
ruta.append(0)
total += dist_matrix[actual][0]

# Mostrar resultados
print("🧭 Ruta óptima con algoritmo del vecino más cercano:\n")
for idx in ruta:
    print("→", nombres[idx])
print(f"\n📏 Distancia total aproximada: {total:.2f} km")

# Crear mapa
mapa = folium.Map(location=[4.65, -74.1], zoom_start=12)

# Agregar marcadores
for i in ruta:
    nombre = nombres[i]
    lat, lon = coordenadas[i]
    folium.Marker([lat, lon], tooltip=nombre).add_to(mapa)

# Agregar rutas reales en el mapa
for i in range(len(ruta) - 1):
    origen = coords_ors[ruta[i]]
    destino = coords_ors[ruta[i+1]]
    try:
        ruta_geojson = client.directions([origen, destino], profile='driving-car', format='geojson')
        folium.GeoJson(ruta_geojson, name="Ruta").add_to(mapa)
    except:
        continue  # en caso de error, sigue con la siguiente ruta

# Mostrar mapa
end_time = time.time()
print(f"\n⏱️ Tiempo de ejecución: {end_time - start_time:.2f} segundos")
mapa


🧭 Ruta óptima con algoritmo del vecino más cercano:

→ Inicio/Fin: Cra 16 A No 28B-33
→ Colegio La Merced
→ Colegio Nicolás Esguerra
→ Colegio INEM Kennedy
→ Colegio Agustiniano Norte
→ Colegio Distrital Usaquén
→ Inicio/Fin: Cra 16 A No 28B-33

📏 Distancia total aproximada: 59.66 km

⏱️ Tiempo de ejecución: 6.33 segundos
